In [ ]:
import sys
sys.path.insert(0, '/src')
import pickle
from data_models import BaseModel
from shared.load_raw_data import join_tables
from pathlib import Path
import boto3
from urllib.parse import urlparse
import pandas as pd

import shap
import re

In [ ]:
prediction_date = '2020-11-23'
client = 'marquis'
facilityid = '26'
s3_folder = 'raw'
env = 'staging'

In [ ]:
# table_list = ['master_patient_lookup','patient_census','patient_rehosps',
#              'patient_progress_notes','patient_diagnosis','patient_vitals','patient_lab_results',
#            'patient_meds','patient_orders','patient_alerts','patient_demographics']
table_list = ['patient_progress_notes']
raw_data_dict = {}
for table in table_list:
    print(f"reading {table}")
    raw_data_dict[table] = pd.read_parquet(
                    f"s3://saiva-{env}-data-bucket/data/{client}/{prediction_date}/{facilityid}/{s3_folder}/{table}.parquet"
                )
# joing with masterpatientid
# raw_data_dict = join_tables(raw_data_dict)
# sort and dedup raw data
# sorting_and_deduping_dataframes(raw_data_dict)

In [ ]:
prog_notes = raw_data_dict['patient_progress_notes']
prog_notes.head()

In [ ]:
prog_notes = prog_notes[
            prog_notes.createddate > max(prog_notes['createddate']) - pd.to_timedelta('14 days')]
prog_notes.head()

In [ ]:
prog_notes[prog_notes.patientid == 1221949]

In [ ]:
# sorted_prog_notes = prog_notes.sort_values(by=['patientid', 'createddate', 'progressnoteid', 'progressnotetype',
#                                      'section', 'sectionsequence', 'notetextorder'])
sorted_prog_notes = prog_notes.sort_values(by=['patientid', 'progressnoteid', 'progressnotetype', 'sectionsequence', 'notetextorder'])
sorted_prog_notes[(sorted_prog_notes.patientid == 1221949) & (sorted_prog_notes.progressnotetype == 'eINTERACT SBAR Summary for Providers')]

In [ ]:
grp_cols = ['patientid', 'createddate', 'progressnoteid', 'progressnotetype', 'section']
progress_notes = (sorted_prog_notes.groupby(grp_cols).agg(
                {'notetext': lambda note_parts: ''.join(note_parts)}).reset_index())
# progress_notes.drop(['progressnoteid', 'section'], axis=1, inplace=True)
progress_notes[(progress_notes.patientid == 1221949) & (progress_notes.progressnotetype == 'eINTERACT SBAR Summary for Providers')]

In [ ]:
progress_notes['clean_note'] = progress_notes['notetext']
progress_notes[(progress_notes.patientid == 1221949) & (progress_notes.progressnotetype == 'eINTERACT SBAR Summary for Providers')]

In [ ]:
list(progress_notes[(progress_notes.patientid == 1221949) & (progress_notes.progressnotetype == 'eINTERACT SBAR Summary for Providers')]['clean_note'][2:3])

In [ ]:
progress_notes.loc[progress_notes.progressnotetype == 'eINTERACT SBAR Summary for Providers', 'test_note'] =  (
    progress_notes.loc[progress_notes.progressnotetype == 'eINTERACT SBAR Summary for Providers'].apply(lambda row: row['clean_note'].split('\n')[0], axis=1))
progress_notes[(progress_notes.patientid == 1221949) & (progress_notes.progressnotetype == 'eINTERACT SBAR Summary for Providers')]

In [ ]:
list(progress_notes[(progress_notes.patientid == 1221949) & (progress_notes.progressnotetype == 'eINTERACT SBAR Summary for Providers')]['clean_note'][2:3])

In [ ]:
list(progress_notes[(progress_notes.patientid == 1221949) & (progress_notes.progressnotetype == 'eINTERACT SBAR Summary for Providers')]['test_note'][2:3])

In [ ]:
progress_notes[(progress_notes.patientid == 1221949) & (progress_notes.progressnotetype != 'eINTERACT SBAR Summary for Providers')]